# 01_Crop_NIFTI.ipynb

This is a manual cropping tool for 3D CT scans. 

This script records the volumn (pixel range) for the region of interest (adrenal glands in this case) and generates `CropArray_CT_NIFTI_full.csv`. `CropArray_CT_NIFTI_full.csv` is then used later for the cropping of the whole dataset in `01_Crop_NIFTI_all.ipynb`.

Required files:
- `DICOM_Data.csv`

Generate:
- `CropArray_CT_NIFTI_full.csv`

In [ ]:
import sys
repo_dir = <PATH TO THIS REPO> # local path to this repo.
sys.path.insert(0, repo_dir)

In [1]:
import os
import pandas as pd
import numpy as np
import math
from os import listdir
import glob
import itertools
import fnmatch
import random
%matplotlib inline
import matplotlib.pylab as plt
import seaborn as sns
import nibabel as nib
import cv2
from tqdm import tqdm
from PIL import Image

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", 100)

# utilitiy files located in the src folder
from src.util_data import listdir_nohidden
from src.util_image import crop_3dimage_n_plot, save_midplane, read_nifti_file, save_crop_3d
from src.util_image import normalise
from src.util_plot_ct import plot_localizer_from_axial, plot_localizer_crop_line
from src.util_plot_ct import plot_3d_ct_axi, plot_3d_ct_sag, plot_3d_ct_cor
from src.util_plot_ct import plot_1_cor, plot_1_axi

In [3]:
# locating the folder paths used in this script.
work_dir = <PATH TO project_dir> # path to the directory of the current project (project master path)
data_folder_path = work_dir + 'CT_data/'
CT_NIFTI_full_path = data_folder_path + "CT_NIFTI_full/"

# load the metadata of the dataset.
df_dicom = pd.read_csv(data_folder_path+"DICOM_Data.csv")

In [5]:
# setup the list to record any fail convertions.
if os.path.exists("CropArray_CT_NIFTI_full.csv"):
    # load the existed crop array and save it to a list
    df_CropArray_CT_NIFTI_full = pd.read_csv("CropArray_CT_NIFTI_full.csv",index_col=0)
    cropped_list = list(set(df_CropArray_CT_NIFTI_full.filename_nii))
else:
    # create a empty cropped_list
    cropped_list = []

In [ ]:
# select which subset of scans to crop.
######################## INPUT VALUES ##########################
patient_condition = "normal" # "normal" or "abnormal"
################################################################

condition_group_path = CT_NIFTI_full_path + patient_condition

# get a list with all scans that are not processed yet.
exist_list = []
for i in range(len(listdir_nohidden(condition_group_path))):
    current_nii_sample = os.path.basename(listdir_nohidden(condition_group_path)[i])
    exist_list.append(current_nii_sample)
current_missing_list = list(set(exist_list) - set(cropped_list))

In [42]:
# Choice which NIFTI to crop
######################## INPUT VALUES ##########################
# Un-comment this if selecting from the current_missing_list
# nii_sample_to_plot = current_missing_list[scan_i] # replace scan_i to select the element in the list
# Un-comment this if selecting from inputing the NIFTI name manually
nii_sample_to_plot = <NAME OF THE NIFTI IMAGE> # crop specific scan by name
################################################################

df_image_metadata = df_dicom[(df_dicom.filename_3d+".nii" == nii_sample_to_plot)]
orig_nii_path = condition_group_path + "/" + nii_sample_to_plot
print(orig_nii_path)
print("Number of slices in this CT scan:", df_image_metadata["numberofslices"].to_string(index=False))

/home/azureuser/cloudfiles/code/Users/sanson.poon/Data_CT_NIFTI/CT_NIFTI_full/normal/00000492.nii
Number of slices in this CT scan:  194


In [ ]:
# Set the cropping boundary (adjust to the ROI)
######################## INPUT VALUES ##########################
square_axi_YN   = "Y"       # the crop on the axial slice to be square [Y/N]
LR_index        = "L"       # left/right part of the patient's body ["L":left or "R":right]
crop_z          = [163,193] # Pixel value: z boundary, cyan boundaries (axial slice number)
crop_x_min      = 185       # Pixel value: minimum pixel value for x boundary. Current set up is for 120 pixel in length for x-axis
y_mid_point     = 256       # patient mid-point (y location of the middle of the backbone) Current set up is for 120 pixel in length for y-axis
################################################################


# To show the crop boundary of the CT scan. Change above input values to adjust boundaries to focus on the region of interest.
crop_x          = [crop_x_min,crop_x_min+120] # Red boundaries
if square_axi_YN == "Y": # square cropped axial slices
    length_crop_x = crop_x[1]-crop_x[0]
    if LR_index == "R":
        crop_y = [y_mid_point-length_crop_x,y_mid_point] # Blue boundaries 
    else:
        crop_y = [y_mid_point,y_mid_point+length_crop_x] # Blue boundaries
else: # set crop y manually
    if LR_index == "R":
        crop_y = [150,275] # Blue boundaries 
    else:
        crop_y = [256,408] # Blue boundaries

# Position of localizer in pixel (localizer slice to show)
# localizer_array = [175, 215, 200]

# Get cropped image according to the set boundaries
crop_array = [crop_x,crop_y,crop_z]
image_3d_cropped, aff_3d, header_3d = crop_3dimage_n_plot(orig_nii_path, crop_array, localizer_array, manual_set_localizer=False)

# Record the current crop array to the data
crop_current_data = {'filename_nii':nii_sample_to_plot, "adrenal_LR":LR_index,'crop_x_min':crop_x[0], 'crop_x_max':crop_x[1], 'crop_y_min':crop_y[0], 'crop_y_max':crop_y[1], 'crop_z_min':crop_z[0], 'crop_z_max':crop_z[1]}
df_crop_current = pd.DataFrame(data=crop_current_data, index=[0])
df_crop = record_crop_array("CropArray_CT_NIFTI_full.csv",crop_current_data)

### df_crop

The generated DataFrame `df_crop` (saved as `CropArray_CT_NIFTI_full.csv`) has the following format:

| filename_nii | adrenal_LR | crop_x_min | crop_x_max | crop_y_min | crop_y_max | crop_z_min | crop_z_max |
|:-------------|-----------:|-----------:|-----------:|-----------:|-----------:|-----------:|-----------:|
|      xxx.nii |          L |        185 |        305 |        256 |        376 |        163 |        193 |
|      xxx.nii |          R |        185 |        305 |        136 |        256 |        163 |        193 |
|      yyy.nii |          L |        150 |        270 |        248 |        360 |         10 |         70 |
|      yyy.nii |          R |        150 |        270 |        128 |        248 |         10 |         70 |

where:
- `filename_nii`: the name of the NIFTI filename_nii
- `adrenal_LR`: Left (L) or right (R) adrenal gland
- `crop_x_min`; `crop_x_max`; `crop_y_min`; `crop_y_max`; `crop_z_min`; `crop_z_max`: the boundaries of the region of interest (w.r.t. pixel values of the original CT scan).

P.S. For this use case: (`crop_x_max` - `crop_x_min`) = (`crop_y_max` - `crop_y_min`) = 120 pixel.


## Plot all cropped axial slices of the CT scan
To check if the whole adrenal glands are covered.

In [ ]:
image_3d_cropped_n = image_3d_cropped
image_3d_cropped_n = normalise(image_3d_cropped,max_n=400,min_n=-200)
plot_3d_ct_axi(image_3d_cropped_n)
del image_3d_cropped_n

## Check cropping progress (w.r.t the dataset)

In [ ]:
#check progress - no. left in normal and abnomal to process (NOT for normal+abnormal).
all_name_list = []
for i in range(len(listdir_nohidden(CT_NIFTI_full_path + "normal"))):
    all_name_list.append(os.path.basename(listdir_nohidden(CT_NIFTI_full_path + "normal")[i]))
for j in range(len(listdir_nohidden(CT_NIFTI_full_path + "abnormal"))):
    all_name_list.append(os.path.basename(listdir_nohidden(CT_NIFTI_full_path + "abnormal")[j]))
print("Total number of normal image:", len(listdir_nohidden(CT_NIFTI_full_path + "normal")))
print("Total number of abnormal image:", len(listdir_nohidden(CT_NIFTI_full_path + "abnormal")))
print("Image processed:", len(set(df_crop["filename_nii"])))

Example output for the progress check:
```
Total number of normal image: 183
Total number of abnormal image: 87
Image processed: 210
```